In [1]:
import sys
import os
import logging
import papermill as pm
import scrapbook as sb
from tempfile import TemporaryDirectory
import numpy as np
import tensorflow.compat.v1 as tf
tf.get_logger().setLevel('ERROR') # only show error messages

if os.path.join('..', '..', 'recommenders') not in sys.path:
    sys.path.append(os.path.join('..', '..', 'recommenders'))

from recommenders.utils.timer import Timer
from recommenders.utils.constants import SEED
from recommenders.models.deeprec.deeprec_utils import (
    prepare_hparams
)
from recommenders.datasets.amazon_reviews import download_and_extract, data_preprocessing
from recommenders.datasets.download_utils import maybe_download


# Locally import the model
from models.deeprec.models.sequential.din import DIN_RECModel as SeqModel


#from recommenders.models.deeprec.models.sequential.nextitnet import NextItNetModel

from recommenders.models.deeprec.io.sequential_iterator import SequentialIterator
#from recommenders.models.deeprec.io.nextitnet_iterator import NextItNetIterator

print("System version: {}".format(sys.version))
print("Tensorflow version: {}".format(tf.__version__))

import pickle as pkl
from recommenders.models.deeprec.deeprec_utils import load_dict

System version: 3.9.7 (default, Sep 16 2021, 13:09:58) 
[GCC 7.5.0]
Tensorflow version: 2.9.0


In [2]:
yaml_file = '../../recommenders/models/deeprec/config/din.yaml'  

In [3]:
EPOCHS = 10
BATCH_SIZE = 400
RANDOM_SEED = SEED  # Set None for non-deterministic result

data_path = os.path.join("..", "..", "tests", "resources", "deeprec", "slirec")

In [4]:

# for test
train_file = os.path.join(data_path, r'train_data')
valid_file = os.path.join(data_path, r'valid_data')
test_file = os.path.join(data_path, r'test_data')
user_vocab = os.path.join(data_path, r'user_vocab.pkl')
item_vocab = os.path.join(data_path, r'item_vocab.pkl')
cate_vocab = os.path.join(data_path, r'category_vocab.pkl')
output_file = os.path.join(data_path, r'output.txt')

reviews_name = 'reviews_Movies_and_TV_5.json'
meta_name = 'meta_Movies_and_TV.json'
reviews_file = os.path.join(data_path, reviews_name)
meta_file = os.path.join(data_path, meta_name)
train_num_ngs = 4 # number of negative instances with a positive instance for training
valid_num_ngs = 4 # number of negative instances with a positive instance for validation
test_num_ngs = 9 # number of negative instances with a positive instance for testing
# sample_rate = 0.01 # sample a small item set for training and testing here for fast example
sample_rate = 1

input_files = [reviews_file, meta_file, train_file, valid_file, test_file, user_vocab, item_vocab, cate_vocab]

if not os.path.exists(train_file):
    download_and_extract(reviews_name, reviews_file)
    download_and_extract(meta_name, meta_file)
    data_preprocessing(*input_files, sample_rate=sample_rate, valid_num_ngs=valid_num_ngs, test_num_ngs=test_num_ngs)
    #### uncomment this for the NextItNet model, because it does not need to unfold the user history
    # data_preprocessing(*input_files, sample_rate=sample_rate, valid_num_ngs=valid_num_ngs, test_num_ngs=test_num_ngs, is_history_expanding=False)

# data_preprocessing(*input_files, sample_rate=sample_rate, valid_num_ngs=valid_num_ngs, test_num_ngs=test_num_ngs)

In [5]:
hparams = prepare_hparams(yaml_file, 
                          embed_l2=0., 
                          layer_l2=0., 
                          learning_rate=0.001,  # set to 0.01 if batch normalization is disable
                          epochs=EPOCHS,
                          batch_size=BATCH_SIZE,
                          show_step=20,
                          MODEL_DIR=os.path.join(data_path, "model/din_dice/"),
                          SUMMARIES_DIR=os.path.join(data_path, "summary/din_dice/"),
                          user_vocab=user_vocab,
                          item_vocab=item_vocab,
                          cate_vocab=cate_vocab,
                          need_sample=True,
                          train_num_ngs=train_num_ngs, # provides the number of negative instances for each positive instance for loss computation.
                          attention_mode="outer_product",
                          activation=['dice', 'dice']
            )

In [6]:
hparams.values()

{'use_entity': True,
 'use_context': True,
 'cross_activation': 'identity',
 'user_dropout': True,
 'dropout': [0.3, 0.3],
 'attention_dropout': 0.0,
 'load_saved_model': False,
 'fast_CIN_d': 0,
 'use_Linear_part': False,
 'use_FM_part': False,
 'use_CIN_part': False,
 'use_DNN_part': False,
 'init_method': 'tnormal',
 'init_value': 0.01,
 'embed_l2': 0.0,
 'embed_l1': 0.0,
 'layer_l2': 0.0,
 'layer_l1': 0.0,
 'cross_l2': 0.0,
 'cross_l1': 0.0,
 'reg_kg': 0.0,
 'learning_rate': 0.001,
 'lr_rs': 1,
 'lr_kg': 0.5,
 'kg_training_interval': 5,
 'max_grad_norm': 2,
 'is_clip_norm': 0,
 'dtype': 32,
 'optimizer': 'adam',
 'epochs': 10,
 'batch_size': 400,
 'enable_BN': True,
 'show_step': 20,
 'save_model': True,
 'save_epoch': 1,
 'write_tfevents': True,
 'train_num_ngs': 4,
 'need_sample': True,
 'embedding_dropout': 0.0,
 'EARLY_STOP': 10,
 'min_seq_length': 1,
 'slots': 5,
 'cell': 'SUM',
 'user_vocab': '../../tests/resources/deeprec/slirec/user_vocab.pkl',
 'item_vocab': '../../tests/r

In [7]:
input_creator = SequentialIterator
#### uncomment this for the NextItNet model, because it needs a special data iterator for training
#input_creator = NextItNetIterator

In [8]:
model = SeqModel(hparams, input_creator, seed=RANDOM_SEED)

with Timer() as train_time:
    model = model.fit(train_file, valid_file, valid_num_ngs=valid_num_ngs) 

# valid_num_ngs is the number of negative lines after each positive line in your valid_file 
# we will evaluate the performance of model on valid_file every epoch
print('Time cost for training is {0:.2f} mins'.format(train_time.interval/60.0))

/home/caozheng/Desktop/code/recommenders/examples/00_quick_start/../../recommenders/models/deeprec/models/base_model.py:724: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  curr_hidden_nn_layer = tf.compat.v1.layers.batch_normalization(
2022-06-05 17:10:59.805629: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-05 17:10:59.810128: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-05 17:10:59.810307: I tensorflow/stream_executor/cuda

step 20 , total_loss: 1.5636, data_loss: 1.5636
step 40 , total_loss: 1.5630, data_loss: 1.5630
step 60 , total_loss: 1.5526, data_loss: 1.5526
step 80 , total_loss: 1.4973, data_loss: 1.4973
step 100 , total_loss: 1.5177, data_loss: 1.5177
step 120 , total_loss: 1.5538, data_loss: 1.5538
step 140 , total_loss: 1.5166, data_loss: 1.5166
step 160 , total_loss: 1.5119, data_loss: 1.5119
step 180 , total_loss: 1.5292, data_loss: 1.5292
step 200 , total_loss: 1.5130, data_loss: 1.5130
step 220 , total_loss: 1.4745, data_loss: 1.4745
step 240 , total_loss: 1.4867, data_loss: 1.4867
step 260 , total_loss: 1.4845, data_loss: 1.4845
step 280 , total_loss: 1.4967, data_loss: 1.4967
step 300 , total_loss: 1.4348, data_loss: 1.4348
step 320 , total_loss: 1.3992, data_loss: 1.3992
step 340 , total_loss: 1.4690, data_loss: 1.4690
step 360 , total_loss: 1.4061, data_loss: 1.4061
step 380 , total_loss: 1.4192, data_loss: 1.4192
step 400 , total_loss: 1.4326, data_loss: 1.4326
step 420 , total_loss: 1

eval valid at epoch 1: auc:0.793,logloss:0.6768,mean_mrr:0.734,ndcg@2:0.6958,ndcg@4:0.7835,ndcg@6:0.8007,group_auc:0.7947
step 20 , total_loss: 0.9912, data_loss: 0.9912
step 40 , total_loss: 0.9293, data_loss: 0.9293
step 60 , total_loss: 0.9967, data_loss: 0.9967
step 80 , total_loss: 0.9945, data_loss: 0.9945
step 100 , total_loss: 1.0453, data_loss: 1.0453
step 120 , total_loss: 0.9906, data_loss: 0.9906
step 140 , total_loss: 0.9479, data_loss: 0.9479
step 160 , total_loss: 1.0351, data_loss: 1.0351
step 180 , total_loss: 0.9580, data_loss: 0.9580
step 200 , total_loss: 1.0197, data_loss: 1.0197
step 220 , total_loss: 0.9481, data_loss: 0.9481
step 240 , total_loss: 0.9707, data_loss: 0.9707
step 260 , total_loss: 1.0223, data_loss: 1.0223
step 280 , total_loss: 0.9398, data_loss: 0.9398
step 300 , total_loss: 0.9515, data_loss: 0.9515
step 320 , total_loss: 0.9649, data_loss: 0.9649
step 340 , total_loss: 0.9810, data_loss: 0.9810
step 360 , total_loss: 0.9883, data_loss: 0.9883


step 3280 , total_loss: 0.8924, data_loss: 0.8924
step 3300 , total_loss: 0.9699, data_loss: 0.9699
eval valid at epoch 2: auc:0.8189,logloss:0.633,mean_mrr:0.7634,ndcg@2:0.7326,ndcg@4:0.8091,ndcg@6:0.8228,group_auc:0.8209
step 20 , total_loss: 0.8894, data_loss: 0.8894
step 40 , total_loss: 0.8872, data_loss: 0.8872
step 60 , total_loss: 0.8528, data_loss: 0.8528
step 80 , total_loss: 0.8498, data_loss: 0.8498
step 100 , total_loss: 0.8382, data_loss: 0.8382
step 120 , total_loss: 0.8428, data_loss: 0.8428
step 140 , total_loss: 0.8226, data_loss: 0.8226
step 160 , total_loss: 0.8480, data_loss: 0.8480
step 180 , total_loss: 0.9250, data_loss: 0.9250
step 200 , total_loss: 0.8606, data_loss: 0.8606
step 220 , total_loss: 0.9100, data_loss: 0.9100
step 240 , total_loss: 0.9012, data_loss: 0.9012
step 260 , total_loss: 0.7607, data_loss: 0.7607
step 280 , total_loss: 0.8310, data_loss: 0.8310
step 300 , total_loss: 0.7963, data_loss: 0.7963
step 320 , total_loss: 0.7325, data_loss: 0.73

step 3240 , total_loss: 0.8808, data_loss: 0.8808
step 3260 , total_loss: 0.8749, data_loss: 0.8749
step 3280 , total_loss: 0.9057, data_loss: 0.9057
step 3300 , total_loss: 0.8635, data_loss: 0.8635
eval valid at epoch 3: auc:0.8284,logloss:0.6861,mean_mrr:0.775,ndcg@2:0.7465,ndcg@4:0.8188,ndcg@6:0.8315,group_auc:0.8306
step 20 , total_loss: 0.7927, data_loss: 0.7927
step 40 , total_loss: 0.8111, data_loss: 0.8111
step 60 , total_loss: 0.8140, data_loss: 0.8140
step 80 , total_loss: 0.8524, data_loss: 0.8524
step 100 , total_loss: 0.7806, data_loss: 0.7806
step 120 , total_loss: 0.7855, data_loss: 0.7855
step 140 , total_loss: 0.7536, data_loss: 0.7536
step 160 , total_loss: 0.8252, data_loss: 0.8252
step 180 , total_loss: 0.7574, data_loss: 0.7574
step 200 , total_loss: 0.7774, data_loss: 0.7774
step 220 , total_loss: 0.7672, data_loss: 0.7672
step 240 , total_loss: 0.7039, data_loss: 0.7039
step 260 , total_loss: 0.7725, data_loss: 0.7725
step 280 , total_loss: 0.7997, data_loss: 0.

...

step 3120 , total_loss: 0.7823, data_loss: 0.7823
step 3140 , total_loss: 0.7075, data_loss: 0.7075
step 3160 , total_loss: 0.7513, data_loss: 0.7513
step 3180 , total_loss: 0.8149, data_loss: 0.8149
step 3200 , total_loss: 0.8097, data_loss: 0.8097
step 3220 , total_loss: 0.7766, data_loss: 0.7766
step 3240 , total_loss: 0.6717, data_loss: 0.6717
step 3260 , total_loss: 0.6865, data_loss: 0.6865
step 3280 , total_loss: 0.8010, data_loss: 0.8010
step 3300 , total_loss: 0.6573, data_loss: 0.6573
eval valid at epoch 6: auc:0.8347,logloss:0.8523,mean_mrr:0.7807,ndcg@2:0.7536,ndcg@4:0.8238,ndcg@6:0.8358,group_auc:0.8353
step 20 , total_loss: 0.6111, data_loss: 0.6111
step 40 , total_loss: 0.6161, data_loss: 0.6161
step 60 , total_loss: 0.6822, data_loss: 0.6822
step 80 , total_loss: 0.6491, data_loss: 0.6491
step 100 , total_loss: 0.7265, data_loss: 0.7265
step 120 , total_loss: 0.7866, data_loss: 0.7866
step 140 , total_loss: 0.6583, data_loss: 0.6583
step 160 , total_loss: 0.6598, data_l

step 3080 , total_loss: 0.7365, data_loss: 0.7365
step 3100 , total_loss: 0.7337, data_loss: 0.7337
step 3120 , total_loss: 0.7146, data_loss: 0.7146
step 3140 , total_loss: 0.6949, data_loss: 0.6949
step 3160 , total_loss: 0.8358, data_loss: 0.8358
step 3180 , total_loss: 0.7386, data_loss: 0.7386
step 3200 , total_loss: 0.7459, data_loss: 0.7459
step 3220 , total_loss: 0.6630, data_loss: 0.6630
step 3240 , total_loss: 0.7371, data_loss: 0.7371
step 3260 , total_loss: 0.7931, data_loss: 0.7931
step 3280 , total_loss: 0.7208, data_loss: 0.7208
step 3300 , total_loss: 0.7402, data_loss: 0.7402
eval valid at epoch 7: auc:0.8361,logloss:0.8996,mean_mrr:0.7833,ndcg@2:0.7567,ndcg@4:0.8259,ndcg@6:0.8378,group_auc:0.8376
step 20 , total_loss: 0.6369, data_loss: 0.6369
step 40 , total_loss: 0.6736, data_loss: 0.6736
step 60 , total_loss: 0.6288, data_loss: 0.6288
step 80 , total_loss: 0.6596, data_loss: 0.6596
step 100 , total_loss: 0.6385, data_loss: 0.6385
step 120 , total_loss: 0.6840, data

step 3040 , total_loss: 0.7484, data_loss: 0.7484
step 3060 , total_loss: 0.6658, data_loss: 0.6658
step 3080 , total_loss: 0.8069, data_loss: 0.8069
step 3100 , total_loss: 0.7356, data_loss: 0.7356
step 3120 , total_loss: 0.6312, data_loss: 0.6312
step 3140 , total_loss: 0.6954, data_loss: 0.6954
step 3160 , total_loss: 0.7290, data_loss: 0.7290
step 3180 , total_loss: 0.7459, data_loss: 0.7459
step 3200 , total_loss: 0.6729, data_loss: 0.6729
step 3220 , total_loss: 0.7228, data_loss: 0.7228
step 3240 , total_loss: 0.7415, data_loss: 0.7415
step 3260 , total_loss: 0.6821, data_loss: 0.6821
step 3280 , total_loss: 0.7131, data_loss: 0.7131
step 3300 , total_loss: 0.7243, data_loss: 0.7243
eval valid at epoch 8: auc:0.8347,logloss:0.9789,mean_mrr:0.7816,ndcg@2:0.7539,ndcg@4:0.8242,ndcg@6:0.8365,group_auc:0.8357
step 20 , total_loss: 0.6788, data_loss: 0.6788
step 40 , total_loss: 0.5983, data_loss: 0.5983
step 60 , total_loss: 0.6500, data_loss: 0.6500
step 80 , total_loss: 0.6450, da

step 3000 , total_loss: 0.6524, data_loss: 0.6524
step 3020 , total_loss: 0.6968, data_loss: 0.6968
step 3040 , total_loss: 0.7295, data_loss: 0.7295
step 3060 , total_loss: 0.6833, data_loss: 0.6833
step 3080 , total_loss: 0.5644, data_loss: 0.5644
step 3100 , total_loss: 0.7564, data_loss: 0.7564
step 3120 , total_loss: 0.7400, data_loss: 0.7400
step 3140 , total_loss: 0.6652, data_loss: 0.6652
step 3160 , total_loss: 0.6115, data_loss: 0.6115
step 3180 , total_loss: 0.7058, data_loss: 0.7058
step 3200 , total_loss: 0.7259, data_loss: 0.7259
step 3220 , total_loss: 0.6702, data_loss: 0.6702
step 3240 , total_loss: 0.7557, data_loss: 0.7557
step 3260 , total_loss: 0.6779, data_loss: 0.6779
step 3280 , total_loss: 0.7235, data_loss: 0.7235
step 3300 , total_loss: 0.7912, data_loss: 0.7912
eval valid at epoch 9: auc:0.8359,logloss:1.0327,mean_mrr:0.783,ndcg@2:0.7557,ndcg@4:0.8252,ndcg@6:0.8375,group_auc:0.8365
step 20 , total_loss: 0.5823, data_loss: 0.5823
step 40 , total_loss: 0.6590,

step 2960 , total_loss: 0.6458, data_loss: 0.6458
step 2980 , total_loss: 0.6806, data_loss: 0.6806
step 3000 , total_loss: 0.6706, data_loss: 0.6706
step 3020 , total_loss: 0.7140, data_loss: 0.7140
step 3040 , total_loss: 0.7453, data_loss: 0.7453
step 3060 , total_loss: 0.6766, data_loss: 0.6766
step 3080 , total_loss: 0.6905, data_loss: 0.6905
step 3100 , total_loss: 0.7347, data_loss: 0.7347
step 3120 , total_loss: 0.6883, data_loss: 0.6883
step 3140 , total_loss: 0.6120, data_loss: 0.6120
step 3160 , total_loss: 0.7504, data_loss: 0.7504
step 3180 , total_loss: 0.6173, data_loss: 0.6173
step 3200 , total_loss: 0.6259, data_loss: 0.6259
step 3220 , total_loss: 0.6590, data_loss: 0.6590
step 3240 , total_loss: 0.6311, data_loss: 0.6311
step 3260 , total_loss: 0.7021, data_loss: 0.7021
step 3280 , total_loss: 0.6560, data_loss: 0.6560
step 3300 , total_loss: 0.6647, data_loss: 0.6647
eval valid at epoch 10: auc:0.8353,logloss:1.1057,mean_mrr:0.7822,ndcg@2:0.7547,ndcg@4:0.8245,ndcg@6

In [9]:
res_syn = model.run_eval(test_file, num_ngs=test_num_ngs)
print(res_syn)

{'auc': 0.8204, 'logloss': 1.3573, 'mean_mrr': 0.6512, 'ndcg@2': 0.5965, 'ndcg@4': 0.6727, 'ndcg@6': 0.7058, 'group_auc': 0.8205}


In [10]:
model_best_trained = SeqModel(hparams, input_creator, seed=RANDOM_SEED)
path_best_trained = os.path.join(hparams.MODEL_DIR, "best_model")
print('loading saved model in {0}'.format(path_best_trained))
model_best_trained.load_model(path_best_trained)

/home/caozheng/Desktop/code/recommenders/examples/00_quick_start/../../recommenders/models/deeprec/models/base_model.py:724: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  curr_hidden_nn_layer = tf.compat.v1.layers.batch_normalization(


loading saved model in ../../tests/resources/deeprec/slirec/model/din_dice/best_model


2022-06-05 18:19:28.711161: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-05 18:19:28.711360: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-05 18:19:28.711486: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-05 18:19:28.711654: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-05 18:19:28.711779: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from S

In [11]:
model_best_trained.run_eval(test_file, num_ngs=test_num_ngs)

{'auc': 0.8221,
 'logloss': 1.0988,
 'mean_mrr': 0.6528,
 'ndcg@2': 0.5977,
 'ndcg@4': 0.6752,
 'ndcg@6': 0.708,
 'group_auc': 0.8229}